## A convolutional autoencoder

This notebook is edited based on the one used in my Google colab.

In [ ]:
import tensorflow as tf
from tensorflow import keras
import os
import numpy as np

In [ ]:
image = tf.keras.utils.image_dataset_from_directory(
    "./drive/MyDrive/Image/", labels=None, label_mode=None,
    class_names=None, color_mode="grayscale", batch_size=1, image_size=(256,
    256), shuffle=True, seed=None, validation_split=None, subset=None,
    interpolation='bilinear', follow_links=False,
    crop_to_aspect_ratio=False)


The convolutional autoencoder

In [ ]:
encoder = keras.models.Sequential()
input1 = keras.layers.Input(shape=[256,256,1],name="input1")
encoder.add(input1)
input2 = keras.layers.Conv2D(16,kernel_size=2,padding="same",activation="selu",strides=2,name="input2")
encoder.add(input2)

input3 = keras.layers.Conv2D(8,kernel_size=2,padding="same",activation="selu",strides=2,name="input3")
encoder.add(input3)

input4 = keras.layers.Conv2D(8,kernel_size=2,padding="same",activation="selu",strides=2,name="input4")
encoder.add(input4)


decoder = keras.models.Sequential()  

input5 = keras.layers.Conv2DTranspose(8,kernel_size=2,strides = 2,padding="same",activation="selu",input_shape=[32,32,8],name="input5")
decoder.add(input5)

input6 = keras.layers.Conv2DTranspose(16,kernel_size=2,strides = 2,padding="same",activation="selu",name="input6")
decoder.add(input6)

input7 = keras.layers.Conv2DTranspose(1,kernel_size=2,strides = 2, padding="same",activation="selu",name="input7")
decoder.add(input7)

  
auto = keras.models.Sequential([encoder, decoder])


In [ ]:
auto.compile(optimizer="adam",loss="binary_crossentropy")
auto.fit(image,epochs=50)

## A simple GAN

In [ ]:
import numpy as np
from tensorflow import keras
generator = keras.models.Sequential([
    keras.layers.Dense(100,activation="selu",input_shape=[30]),
    keras.layers.Dense(150,activation="selu"),
    keras.layers.Dense(256*256,activation="sigmoid"),
    keras.layers.Reshape([256,256,1])
    ])
generator.summary()

In [ ]:
discriminator = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[256,256,1]),
    keras.layers.Dense(100,activation="selu"),
    keras.layers.Dense(150,activation="selu"),
    keras.layers.Dense(1,activation="sigmoid")
])
discriminator.summary()

In [ ]:
gan = keras.models.Sequential(generator,discriminator)
discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")
discriminator.trainable = False
gan.compile(loss="binary_crossentropy", optimizer="rmsprop")

In [ ]:
def train(gan,image,batch_size,generator,discriminator,n_epochs = 50):
  
  for epoch in range(n_epochs):
    for batch in image:
      noise = np.random.normal(size=[batch_size,30])
      generate =generator(noise)
      fake_and_real = np.concatenate([generate,batch],axis = 0)
      y1 = np.array([[0.]]*batch_size+[[1.]]*batch_size)

      discriminator.trainable = True
      discriminator.train_on_batch(fake_and_real, y1)
      noise = np.random.normal(size=[batch_size,30])
      y2 = np.array([[1.]]*batch_size)
      discriminator.trainable = False
      gan.train_on_batch(noise,y2)

train(gan,image,1,generator,discriminator)